# Création du Dataframe + initialisation de la Data

In [3]:
import pandas as pd
import json
from numpyencoder import NumpyEncoder

# Read csv file
Df = pd.read_csv('./planning_Csv/planningTestV2.csv', sep=';')
# Filter columns and rows
Df = Df[['LUNDI', 'MARDI', 'MERCREDI', 'JEUDI', 'VENDREDI']]
Df.reset_index()
Df.fillna(0, inplace=True)
Df

Index(['LUNDI', 'MARDI', 'MERCREDI', 'JEUDI', 'VENDREDI', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'],
      dtype='object')


,LUNDI,MARDI,MERCREDI,JEUDI,VENDREDI
0,30-janv,31-janv,01-févr,02-févr,03-févr
1,8h-12h15,8h-12h15,8h-12h15,8h-12h15,8h-12h15
2,0,0,0,CONCEP.LOGICIELLE/BIG DATA,CONCEP.LOGICIELLE/BIG DATA
3,Introduction,Introduction,Introduction,Administration Système,0
4,Administration Système,Administration Système,Administration Système,0,0
...,...,...,...,...,...
67,0,Optimisation numérique et problème inverse,T.DUPONT,0,Pas de cours
68,0,0,Salle I304,0,0
69,0,14h45 - 17h45,0,0,0
70,0,E.VAN REETH /S.MAZAURIC,0,0,0


## On récupère la liste de toutes les matière/ prof/ majeurs dans LstSemaine

In [32]:
def createSchedule(Df,Data):
    DicoJour = {}
    IsAprem = False
    LstSemaine = []
    # parcour des jours pour séparer matin et aprem
    for obj in Df :
        LstMatin = []
        LstAprem = []
        LstTemp = []
        # parcour des cases non nulles 
        for case in Df[obj] :
            if case != 0 :
                LstTemp.append(case)

        # parcour des cases non nulles d'une journée pour la séparation
        for index,value in enumerate(LstTemp) :
            if index == 0 :
                # on ajoute le jour
                DicoJour["jour"] = obj +"-" + value
            else :
                if value in {'13h-14h30','13h30-17h45','13h10-18h', '13h30-15h30', '15h45-17h45', '13h30 - 14h30'}:
                  IsAprem = True
                elif value in {'8h-12h15', '8h-10h', '10h15-12h15', '8h- 10h', }:
                    IsAprem = False
                
                if IsAprem :
                    LstAprem.append(value)
                else :
                    LstMatin.append(value)

        DicoJour["Matin"] = LstMatin.copy()
        DicoJour["Aprem"] = LstAprem.copy()
        LstSemaine.append(DicoJour.copy())
    return LstSemaine

LstSemaine = createSchedule(Df,Data)
print(LstSemaine)
df_semaine = pd.DataFrame( LstSemaine)
df_semaine
# df_semaine.to_json('planning_semaineMatin_Aprem.json',orient='records', indent=4)

[{'jour': 'LUNDI-30-janv', 'Matin': ['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G102 ', 'Grp B  Salle G108 ', 'Grp C  Salle I304 ', 'Grp D  Salle G103'], 'Aprem': ['13h10-18h', 'LV', '(Voir planning)']}, {'jour': 'MARDI-31-janv', 'Matin': ['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G102 ', 'Grp B  Salle I300', 'Grp C  Salle I207 ', 'Grp D  Salle G103'], 'Aprem': ['13h30-17h45', 'CONCEP.LOGICIELLE/BIG DATA', 'Bases de Données ', 'J.CHEVALIER/M.LAGARRIGUE', 'Salles  G102- G103', 'ROBOTIQUE', 'Administration système et gestion de codes', 'F.JUMEL', 'Salle G108', 'ELECTRONIQUE ET SYST EMB', 'Pas de cours', 'INFRA DES RESEAUX ', 'Architecture des réseaux locaux LAN', 'N.DANIEL ', 'Salle B121', 'R.FANTAUZZI', 'Salle I304', 'IMAGE ', 'Rentrée ', '13h30 - 14h30', 'Optimisation numérique et problème inverse', '14h45 - 17h45', 'E.VAN REETH /S.MAZAURIC', 'Salle F002']}, {'jour': 'MERCREDI-01-févr', 'Matin': ['8h-12h15', 'Intr

,jour,Matin,Aprem
0,LUNDI-30-janv,"[8h-12h15, Introduction, Administration Systèm...","[13h10-18h, LV, (Voir planning)]"
1,MARDI-31-janv,"[8h-12h15, Introduction, Administration Systèm...","[13h30-17h45, CONCEP.LOGICIELLE/BIG DATA, Base..."
2,MERCREDI-01-févr,"[8h-12h15, Introduction, Administration Systèm...","[13h30-17h45, CONCEP.LOGICIELLE/BIG DATA, Admi..."
3,JEUDI-02-févr,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, Adminis...",[]
4,VENDREDI-03-févr,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, pas de ...","[13h30-17h45, CONCEP.LOGICIELLE/BIG DATA, Base..."


## On filtre les Majeurs par jour/ Demi-journée et majeurs ou tronc commun

In [33]:
def filtreMaj(LstSemaine) :
    Semaine =  {}
    for jour in LstSemaine :
        Majeur = "Pour tous" #on initialise le majeur à tous
        Mat= {"CONCEP.LOGICIELLE/BIG DATA":[],
        "ROBOTIQUE":[],
        "ELECTRONIQUE ET SYST EMB":[],
        "INFRA DES RESEAUX":[],
        "IMAGE":[],
        "Pour tous":[]} #on initialise les majeurs
        Semaine[jour["jour"]] = {}
        for Case in jour["Matin"] : #on parcours les cours du matin et on les ajoute au majeur correspondant 
            if Case == "CONCEP.LOGICIELLE/BIG DATA" :
                Majeur = "CONCEP.LOGICIELLE/BIG DATA"
            elif Case == "ROBOTIQUE" :
                Majeur = "ROBOTIQUE"
            elif Case == "ELECTRONIQUE ET SYST EMB" :
                Majeur = "ELECTRONIQUE ET SYST EMB"
            elif Case == "INFRA DES RESEAUX " :
                Majeur = "INFRA DES RESEAUX"
            elif Case == "IMAGE " :
                Majeur = "IMAGE"
            Mat[Majeur].append(Case)
            
        Semaine[jour["jour"]]["Matin"] = Mat
        Majeur = "Pour tous" #on remet le majeur par défaut
        Aprem= {"CONCEP.LOGICIELLE/BIG DATA":[],
        "ROBOTIQUE":[],
        "ELECTRONIQUE ET SYST EMB":[],
        "INFRA DES RESEAUX":[],
        "IMAGE":[],
        "Pour tous":[]} #on remet le majeur par défaut
        for Case in jour["Aprem"] : #on parcours les cours de l'aprem et on les ajoute au majeur correspondant
            if Case == "CONCEP.LOGICIELLE/BIG DATA" :
                Majeur = "CONCEP.LOGICIELLE/BIG DATA"
            elif Case == "ROBOTIQUE" :
                Majeur = "ROBOTIQUE"
            elif Case == "ELECTRONIQUE ET SYST EMB" :
                Majeur = "ELECTRONIQUE ET SYST EMB"
            elif Case == "INFRA DES RESEAUX " :
                Majeur = "INFRA DES RESEAUX"
            elif Case == "IMAGE " :
                Majeur = "IMAGE"
            
            Aprem[Majeur].append(Case)
        Semaine[jour["jour"]]["Aprem"] = Aprem
    return Semaine
Planning = filtreMaj(LstSemaine)
print(Planning)

{'LUNDI-30-janv': {'Matin': {'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE': [], 'ELECTRONIQUE ET SYST EMB': [], 'INFRA DES RESEAUX': [], 'IMAGE': [], 'Pour tous': ['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G102 ', 'Grp B  Salle G108 ', 'Grp C  Salle I304 ', 'Grp D  Salle G103']}, 'Aprem': {'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE': [], 'ELECTRONIQUE ET SYST EMB': [], 'INFRA DES RESEAUX': [], 'IMAGE': [], 'Pour tous': ['13h10-18h', 'LV', '(Voir planning)']}}, 'MARDI-31-janv': {'Matin': {'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE': [], 'ELECTRONIQUE ET SYST EMB': [], 'INFRA DES RESEAUX': [], 'IMAGE': [], 'Pour tous': ['8h-12h15', 'Introduction', 'Administration Système', 'G.MOREL', 'Grp A  Salle G102 ', 'Grp B  Salle I300', 'Grp C  Salle I207 ', 'Grp D  Salle G103']}, 'Aprem': {'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICIELLE/BIG DATA', 'Bases de Données ', 'J.CHEVALIER/M.LAGARRIGUE', 'Salles  G102- G103'], 'ROBOTIQUE': ['ROBOTIQUE', 'Administr

In [34]:
with open('./Output_Json/Datatest.json', 'w+') as f:
    json.dump(Planning, f, indent=4, cls=NumpyEncoder)

